# Load packages and data

In [1]:
## conda activate r4
## packages for data table processing

#IRkernel::installspec()

## main Seurat package snRNA-seq packages
library(Seurat)
library(SeuratDisk)
library(SeuratWrappers)
library(future)

library(here)
library(tidyverse)

library(dplyr)

library(caret)
library(cluster)

#install.packages('hash')
library(hash)
library(viridis)

library(ggpubr)
library(ggplot2)
options(repr.plot.width=30, repr.plot.height=10)

ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F)
options(repr.plot.width=11, repr.plot.height=8.5)

#######################################################
# 0) Seurat uses the future package for parallelization
## set to be parallel over 28 cores
plan("multicore", workers = 28)
options(future.globals.maxSize = 100 * 1024^3)
options(future.rng.onMisuse = 'ignore')

Warning message:
“package ‘Seurat’ was built under R version 4.1.3”
Attaching SeuratObject

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Warning message:
“package ‘future’ was built under R version 4.1.3”
Warning message:
“package ‘here’ was built under R version 4.1.3”
here() starts at /projects/pfenninggroup/singleCell/Macaque_SealDorsalHorn_snATAC-seq

── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Warning message:
“package ‘tibble’ was built under R version 4.1.3”
Warning message:
“package ‘dplyr’ was built under R version 4.1.3”
── Conflicts ─────────────────────────────────────────────

In [2]:
filename = "/projects/pfenninggroup/singleCell/Macaque_SealDorsalHorn_snATAC-seq/data/tidy_data/rdas/macaque_rna_dh_conserved_names.h5seurat"
macaque_rna_dh = LoadH5Seurat(filename)
DefaultAssay(macaque_rna_dh) <- 'SCT'

# human all neurons
human_refined_rna = LoadH5Seurat('/projects/pfenninggroup/singleCell/Yadav2023_human_spinal_cord_snRNA-seq/data/Seurat_objects/Yadav2023_refined_neuron_SeuratObj_N7.h5Seurat')

# mouse
filename = "/home/vijaykic/BaDoi/data/tidy_data/rdas/final_cluster_assignment.h5seurat"
mouse_rna = LoadH5Seurat(filename)
mouse_rna = mouse_rna[, mouse_rna$dataset == 'Sathyamurthy']
mouse_rna = mouse_rna[,mouse_rna$coarse_cell_types %in% c('Excit','Inhib','CSF','MN')]

Idents(mouse_rna) <- 'final_cluster_assignment'
DefaultAssay(mouse_rna) = 'RNA'

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Initializing SCT with data

Adding counts for SCT

Adding scale.data for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Initializing alra with data

Adding scale.data for alra

Adding miscellaneous information for alra

Adding reduction scanorama

Adding cell embeddings for scanorama

Adding miscellaneous information for scanorama

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Initializing SCT with data

Adding 

In [11]:
### MUST CHANGE MOUSE GENE NAMES

### mouse genes to human genes
convertHumanGeneList <- function(x){
  file = '/home/bnphan/resources/genomes/GRCh38.p13/ENSEMBL_GRCh38.p13_genes_to_Orthologous_mouse_genes.tsv'
  orthologs = read_tsv(file, show_col_types = FALSE) %>% 
    rename_with(make.names) %>% 
    dplyr::filter(Mouse.homology.type== 'ortholog_one2one') %>% 
    dplyr::filter(Mouse.gene.name %in% x) %>% 
    dplyr::select(Mouse.gene.name, Gene.name)
  
  # Print the first 6 genes found to the screen
  print(head(orthologs))
  return(orthologs)
}
# RenameGenesSeurat from https://github.com/satijalab/seurat/issues/1049
RenameGenesSeurat <- function(obj, newnames) {
  # Replace gene names in different slots of a Seurat object. Run this before integration.
  # It only changes obj@assays$RNA@counts, @data and @scale.data.
  print("Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.")
  RNA <- obj@assays$RNA

  if (nrow(RNA) == length(newnames)) {
    if (length(RNA@counts)) RNA@counts@Dimnames[[1]]            <- newnames
    if (length(RNA@data)) RNA@data@Dimnames[[1]]                <- newnames
    if (length(RNA@scale.data)) RNA@scale.data@Dimnames[[1]]    <- newnames
  } else {"Unequal gene sets: nrow(RNA) != nrow(newnames)"}
  obj@assays$RNA <- RNA
  return(obj)
}

## switch out human gene names w/ orthologous mouse gene names, based on ENSEMBL
MmtoHg = convertHumanGeneList(rownames(mouse_rna)) %>% deframe()
mouse_rna = mouse_rna[names(MmtoHg[!is.na(MmtoHg)]), ]
mouse_rna = RenameGenesSeurat(mouse_rna, newnames = MmtoHg[rownames(mouse_rna)])
mouse_rna = mouse_rna[!is.na(rownames(mouse_rna)), ]
head(rownames(mouse_rna))

# A tibble: 6 × 2
  Mouse.gene.name Gene.name
  <chr>           <chr>    
1 Frg1            NA       
2 Usp9y           USP9Y    
3 Usp9y           USP9Y    
4 Usp9y           USP9Y    
5 Usp9y           USP9Y    
6 Usp9y           USP9Y    
[1] "Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data."


Usp9y    Kdm5d      Sry    Ddx3y      Uty   Tmsb4x 
 "USP9Y"  "KDM5D"    "SRY"  "DDX3Y"    "UTY" "TMSB4Y"

In [12]:
# Run this AFTER fixing gene names
mouse_rna <- SCTransform(mouse_rna)
mouse_rna <- RunPCA(mouse_rna, npcs = 30, features = rownames(mouse_rna))
mouse_rna <- RunUMAP(mouse_rna, dims = 1:30)

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 13233 by 5464

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells



  |======================================================================| 100%


Found 42 outliers - those will be ignored in fitting/regularization step


Second step: Get residuals using fitted parameters for 13233 genes



  |======================================================================| 100%


Computing corrected count matrix for 13233 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 2.031351 mins

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix

Set default assay to SCT

Warning message in PrepDR(object = object, features = features, verbose = verbose):
“The following 10233 features requested have not been scaled (running reduction without them): USP9Y, PARP4, TMPRSS15, MIPEP, KATNAL1, URB1, KPNA3, CAB39L, MRPS6, DNAJC15, CHAF1B, GPR12, WASF3, SCAF4, POSTN, C21orf91, MORC3, FNDC3A, PAXBP1, C21orf62, AKAP11, MZT1, PROSER1, ITSN1, PIBF1, BORA, GTF3A, ELF1, FREM2, TBC1D4, UFM1, UBAC2, RFC3, POU4F1, NHLRC3, IPO5, ATP5PO, DIAPH3, BACH1, ETS2, USP12, WBP4, IMPA2, SLC7A1, N6AMT1, COG3, YES1, C2CD2, LRCH1, ZBTB21, CDADC1, CBR3, RUNX1, NDUFV2, SLX4IP, MPPE1, NKX2-2, BMP2, COG6, BTBD3, TMEM230, BTG3, SUMO3, SLITRK6, MRPL39, SAMSN1, XRN2, SMIM11, SIM2, SNRPB2, ZMYM5, TEX26, RRP1B, CST7, APMAP, PDXK, SLC14A2, RALBP1, DSTN, STK35, SCRT2, NCOA6, SS18, CSTB, RRP1, A

# integrate three species: macaque DH, human neurons, mouse neurons

In [14]:
### Try to integrate all three datasets at the same time instead of sequentially
###
features <- SelectIntegrationFeatures(  object.list = list(human_refined_rna, macaque_rna_dh, mouse_rna)  )

## Find anchors and do integration
mac_human_mouse.anchors <- FindIntegrationAnchors(object.list = list(human_refined_rna, macaque_rna_dh, mouse_rna), anchor.features = features)
mac_human_mouse.combined <- IntegrateData(anchorset = mac_human_mouse.anchors,normalization.method = "SCT")
DefaultAssay(mac_human_mouse.combined) <- "integrated"

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 3305 anchors

Filtering anchors

	Retained 2460 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 4766 anchors

Filtering anchors

	Retained 2088 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 5357 anchors

Filtering anchors

	Retained 2548 anchors

Merging dataset 1 into 2

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 2 1 into 3

Warning message:
“Attempting to merge an SCTAssay with another Assay type 
Converting all to standard Assay objects.”
Extracting anchors for merged samples

Warning message:
“Attempting to merge an SCTAssay with another Assay type 
Converting all to standard Assay objects.”
Finding integration vectors

Finding integration vector weights

Integrating d

In [15]:
# Run the standard workflow for visualization and clustering
mac_human_mouse.combined <- ScaleData(mac_human_mouse.combined, verbose = FALSE)
mac_human_mouse.combined <- RunPCA(mac_human_mouse.combined, npcs = 30, verbose = FALSE)
mac_human_mouse.combined <- RunUMAP(mac_human_mouse.combined, reduction = "pca", dims = 1:30, return.model = TRUE)
mac_human_mouse.combined <- FindNeighbors(mac_human_mouse.combined, reduction = "pca", dims = 1:30)
mac_human_mouse.combined <- FindClusters(mac_human_mouse.combined, resolution = 0.5)

UMAP will return its model

16:52:53 UMAP embedding parameters a = 0.9922 b = 1.112

16:52:53 Read 8761 rows and found 30 numeric columns

16:52:53 Using Annoy for neighbor search, n_neighbors = 30

16:52:53 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

16:52:54 Writing NN index file to temp file /tmp/RtmpvtZffh/file1f99f2134b072

16:52:54 Searching Annoy index using 28 threads, search_k = 3000

16:52:56 Annoy recall = 100%

16:52:57 Commencing smooth kNN distance calibration using 28 threads
 with target n_neighbors = 30

16:52:58 Initializing from normalized Laplacian + noise (using irlba)

16:52:58 Commencing optimization for 500 epochs, with 386948 positive edges

16:53:30 Optimization finished

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 8761
Number of edges: 356848

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9304
Number of communities: 18
Elapsed time: 0 seconds


In [17]:
mac_human_mouse.combined$species = ifelse(is.na(mac_human_mouse.combined$conserved_marker_nicknames), 'notmacaque', 'macaque')
mac_human_mouse.combined$species = ifelse(is.na(mac_human_mouse.combined$final_cluster_assignment), mac_human_mouse.combined$species, 'mouse')
mac_human_mouse.combined$species = ifelse(is.na(mac_human_mouse.combined$subtype_annotation), mac_human_mouse.combined$species, 'human')

mac_human_mouse.combined$integrated_celltype = ifelse(is.na(mac_human_mouse.combined$conserved_marker_nicknames), '', mac_human_mouse.combined$conserved_marker_nicknames)
mac_human_mouse.combined$integrated_celltype = ifelse(is.na(mac_human_mouse.combined$final_cluster_assignment), mac_human_mouse.combined$integrated_celltype, mac_human_mouse.combined$final_cluster_assignment)
mac_human_mouse.combined$integrated_celltype = ifelse(is.na(mac_human_mouse.combined$subtype_annotation), mac_human_mouse.combined$integrated_celltype, mac_human_mouse.combined$subtype_annotation)


In [44]:
# Name NA's as Other Species
mac_human_mouse.combined$subtype_annotation[is.na(mac_human_mouse.combined$subtype_annotation)] = 'Other Species'
mac_human_mouse.combined$conserved_marker_nicknames[is.na(mac_human_mouse.combined$conserved_marker_nicknames)] = 'Other Species'
mac_human_mouse.combined$final_cluster_assignment[is.na(mac_human_mouse.combined$final_cluster_assignment)] = 'Other Species'

In [70]:
library(RColorBrewer)
colors_mouse = c('grey90', DiscretePalette(36), brewer.pal(n=12,"Set3"), brewer.pal(n=8,"Set2"), brewer.pal(n=9,"Set1"), 'grey70')

In [73]:
save_integration_figs = TRUE

ordered_macaque_names = c('Exc-BNC2','Exc-NMUR2', 'Exc-NMU', 'Exc-TAC3','Exc-LMO3', 'Exc-SKOR2', 'Exc-MAF', 'Exc-MAFA', 
                  'Exc-NTS', 'Exc-SNTB1', 'Exc-PBX3', 'Inh-CACNA2D3', 'Inh-SORCS1', 'Inh-PDZD2', 'Inh-NPY', 'Inh-MEF2C','Inh-NXPH1','Inh-PDYN')
ordered_human_names = c('Ex-Dorsal-1','Ex-Dorsal-2','Ex-Dorsal-3','Ex-Dorsal-4','Ex-Dorsal-5','Ex-Dorsal-6',
                        'Ex-Dorsal-7','Ex-Dorsal-8','Ex-Dorsal-9','Ex-Dorsal-10','Ex-Dorsal-11','Ex-Dorsal-12',
                        'Inh-Dorsal-1','Inh-Dorsal-2','Inh-Dorsal-3','Inh-Dorsal-4','Inh-Dorsal-5','Inh-Dorsal-6',
                        'Inh-Dorsal-7','Inh-Dorsal-8','Inh-Dorsal-9')
ordered_mouse_names = sort(unique(mac_human_mouse.combined$final_cluster_assignment))
ordered_mouse_names = ordered_mouse_names[1:length(ordered_mouse_names)-1]

colors_mouse = c('grey90', DiscretePalette(36), brewer.pal(n=12,"Set3"), brewer.pal(n=8,"Set2"), brewer.pal(n=9,"Set1"), 'grey70')
colors_human = c('grey90', DiscretePalette(length(unique(mac_human_mouse.combined$subtype_annotation))) )
colors_macaque = c('grey90','#0075DC','#993F00','#4C005C','#191919','#005C31','#2BCE48','#FFCC99','#808080','#94FFB5','#8F7C00','#9DCC00',
         '#C20088','#003380','#FFA405','#5EF1F2','#FFA8BB','#426600','#FF0010')
order_list = list(ordered_macaque_names, ordered_human_names, ordered_mouse_names)
col_list = list(colors_macaque, colors_human, colors_mouse)
ident_list = c("conserved_marker_nicknames", "subtype_annotation","final_cluster_assignment")
title_list = c("Macaque", "Human", "Mouse")
widths = c(5,6,7.5)
label_yesno = c(TRUE,TRUE, TRUE)

for (ii in 1:3){
    

    Idents(object = mac_human_mouse.combined) <- ident_list[ii]

    
    p <- DimPlot(mac_human_mouse.combined, reduction = "umap", order = order_list[[ii]], cols = col_list[[ii]],
              group.by = 'ident', label = label_yesno[ii], pt.size = .1,
              label.size = 1.8, repel = TRUE, na.value = "grey90", raster = FALSE) + theme(axis.text.y=element_blank(), axis.ticks.y=element_blank(), axis.title.y=element_blank(),
                                                                        axis.ticks.x=element_blank(),axis.text.x=element_blank(),axis.title.x=element_text(size = 9),
                                                                        plot.title = element_text(size = 12),
                                                                         legend.position = "right", legend.text=element_text(size=5)) + 
                labs(title=title_list[ii], x ="UMAP 1")

    if (save_integration_figs){
        #tiff(here(paste0("figures/exploratory/arokiaraj_paper/",title_list[ii],"_integrated_",Sys.Date(),".tiff")), units="in", width=2.5, height=3, res=300)
        #png(here(paste0("figures/exploratory/arokiaraj_paper/",title_list[ii],"_integrated_",Sys.Date(),".png")), units="px", width=2.5*72, height=3*72)
        pdf(here(paste0("figures/exploratory/find_pbx3/",title_list[ii],"_integrated_",Sys.Date(),".pdf")), width = widths[ii], height = 4.5)
    
        print(p)
        dev.off()
        }

}

Warning message:
“ggrepel: 28 unlabeled data points (too many overlaps). Consider increasing max.overlaps”


# Marker Genes from Glut11-14 experiment

In [20]:
markers = c('GPR83', 'TAC1', 'TACR1', 'GRPR', 'MAF', 'CPNE3','NMUR2', 'RREB1')

In [25]:
##### Markers
Idents(human_refined_rna) = 'subtype_annotation'

png(filename=here('figures/exploratory/find_pbx3/unscaled_markers_mouse_excit.png'),
   width=7,height=3,units="in",res=1200)

DoHeatmap(object = mouse_rna[,mouse_rna$coarse_cell_types %in% c('Excit')], features = markers,slot = 'data', size = 2, angle=90) + scale_fill_viridis() + ggtitle('') +
theme(plot.title=element_text(family='', face='bold', colour='black', size=12), axis.text=element_text(size=10)) + guides(colour=FALSE)

dev.off()

png(filename=here('figures/exploratory/find_pbx3/unscaled_markers_mouse_rest.png'),
   width=7,height=3,units="in",res=1200)

DoHeatmap(object = mouse_rna[,mouse_rna$coarse_cell_types %in% c('CSF', 'Inhib','MN')], features = markers,slot = 'data', size = 2, angle=90) + scale_fill_viridis() + ggtitle('') +
theme(plot.title=element_text(family='', face='bold', colour='black', size=12), axis.text=element_text(size=10)) + guides(colour=FALSE)

dev.off()

png(filename=here('figures/exploratory/find_pbx3/unscaled_markers_macaque.png'),
   width=7,height=3,units="in",res=1200)

DoHeatmap(object = macaque_rna_dh, features = markers, slot = 'data', size = 2, angle=90) + scale_fill_viridis() + ggtitle('') +
theme(plot.title=element_text(family='', face='bold', colour='black', size=12), axis.text=element_text(size=10)) + guides(colour=FALSE)

dev.off()

png(filename=here('figures/exploratory/find_pbx3/unscaled_markers_human.png'),
   width=7,height=3,units="in",res=1200)

DoHeatmap(object = human_refined_rna, features = markers, slot = 'data', size = 2, angle=90) + scale_fill_viridis() + ggtitle('') +
theme(plot.title=element_text(family='', face='bold', colour='black', size=12), axis.text=element_text(size=10)) + guides(colour=FALSE)

dev.off()

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


png 
  2

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


png 
  2

Warning message in DoHeatmap(object = macaque_rna_dh, features = markers, slot = "data", :
“The following features were omitted as they were not found in the data slot for the SCT assay: GPR83”
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


png 
  2

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


png 
  2

In [27]:
# just cell types of interest


Idents(human_refined_rna) = 'subtype_annotation'

png(filename=here('figures/exploratory/find_pbx3/unscaled_markers_mouse_target.png'),
   width=7,height=3,units="in",res=1200)

DoHeatmap(object = mouse_rna[,mouse_rna$final_cluster_assignment %in% c('Excit.12','Excit.13','Excit.14','Excit.15','Excit.16','Excit.20','Excit.25','Excit.30')], features = markers,slot = 'data', size = 2, angle=90) + scale_fill_viridis() + ggtitle('') +
theme(plot.title=element_text(family='', face='bold', colour='black', size=12), axis.text=element_text(size=10)) + guides(colour=FALSE)

dev.off()


Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


png 
  2

# integrate just macaque and human

In [ ]:
### JUST MACAQUE AND HUMAN
###

features <- SelectIntegrationFeatures(  object.list = list(macaque_rna_dh, human_refined_rna)  )

## Find anchors and do integration
mac_human.anchors <- FindIntegrationAnchors(object.list = list(human_refined_rna, macaque_rna_dh), anchor.features = features)
mac_human.combined <- IntegrateData(anchorset = mac_human.anchors,normalization.method = "SCT")
DefaultAssay(mac_human.combined) <- "integrated"

# Run the standard workflow for visualization and clustering
mac_human.combined <- ScaleData(mac_human.combined, verbose = FALSE)
mac_human.combined <- RunPCA(mac_human.combined, npcs = 30, verbose = FALSE)
mac_human.combined <- RunUMAP(mac_human.combined, reduction = "pca", dims = 1:30, return.model = TRUE)
mac_human.combined <- FindNeighbors(mac_human.combined, reduction = "pca", dims = 1:30)
mac_human.combined <- FindClusters(mac_human.combined, resolution = 0.5)

mac_human.combined$species = ifelse(is.na(mac_human_mouse.combined$conserved_marker_nicknames), 'notmacaque', 'macaque')
mac_human.combined$species = ifelse(is.na(mac_human_mouse.combined$subtype_annotation), mac_human_mouse.combined$species, 'human')

mac_human.combined$integrated_celltype = ifelse(is.na(mac_human.combined$conserved_marker_nicknames), '', mac_human.combined$conserved_marker_nicknames)
mac_human.combined$integrated_celltype = ifelse(is.na(mac_human.combined$subtype_annotation), mac_human.combined$integrated_celltype, mac_human_mouse.combined$subtype_annotation)


Scaling features for provided objects

Finding all pairwise anchors

Warning message in RunCCA.Seurat(object1 = object.1, object2 = object.2, assay1 = "ToIntegrate", :
“Fewer than 50 features used as input for CCA.”
Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 5300 anchors

Warning message:
“Requested number is larger than the number of available items (7). Setting to 7.”
Warning message:
“Requested number is larger than the number of available items (7). Setting to 7.”
Warning message:
“Requested number is larger than the number of available items (7). Setting to 7.”
Warning message:
“Requested number is larger than the number of available items (7). Setting to 7.”
Warning message:
“Requested number is larger than the number of available items (7). Setting to 7.”
Warning message:
“Requested number is larger than the number of available items (7). Setting to 7.”
Warning message:
“Requested number is larger than the number of available items (7). Setting 

In [51]:
save_integration_figs = TRUE


ident_list = c("conserved_marker_nicknames", "subtype_annotation")
title_list = c("Macaque", "Human")
label_yesno = c(TRUE,TRUE)

for (ii in 1:2){
    
    Idents(object = mac_human.combined) <- ident_list[ii]
    
    p <- DimPlot(mac_human.combined, reduction = "umap",
              group.by = 'ident', label = label_yesno[ii], pt.size = .1,
              label.size = 1.8, repel = TRUE, na.value = "grey90", raster = FALSE) + theme(axis.text.y=element_blank(), axis.ticks.y=element_blank(), axis.title.y=element_blank(),
                                                                        axis.ticks.x=element_blank(),axis.text.x=element_blank(),axis.title.x=element_text(size = 9),
                                                                        plot.title = element_text(size = 12),
                                                                         legend.position = "right", legend.text=element_text(size=6)) + 
                labs(title=title_list[ii], x ="UMAP 1")

    if (save_integration_figs){
        #tiff(here(paste0("figures/exploratory/arokiaraj_paper/",title_list[ii],"_integrated_",Sys.Date(),".tiff")), units="in", width=2.5, height=3, res=300)
        #png(here(paste0("figures/exploratory/arokiaraj_paper/",title_list[ii],"_integrated_",Sys.Date(),".png")), units="px", width=2.5*72, height=3*72)
        pdf(here(paste0("figures/exploratory/find_PBX_",title_list[ii],"_integrated_",Sys.Date(),".pdf")), width = 5, height = 4)
    
        print(p)
        dev.off()
        }


}


Warning message:
“ggrepel: 9 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
